In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoProcessor
import json
import torch
from open_flamingo import create_model_and_transforms
import open_clip
from PIL import Image

/opt/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

In [2]:
# Step 1: Load and preprocess dataset
def load_and_clean_dataset():
    dataset = load_dataset("ivelin/ui_refexp_saved")
    cleaned_data = []

    for entry in dataset["validation"]:
        # Check if entry is a string and attempt to convert to dict
        if isinstance(entry, str):
            try:
                entry = json.loads(entry)
            except json.JSONDecodeError:
                print(f"Warning: Failed to decode JSON string: {entry}")
                continue  # Skip this entry if decoding fails

        cleaned_entry = {
            "image": entry.get("image"),  
            "image_id": entry.get("image_id"),
            "image_file_path": entry.get("image_file_path"),
            "prompt": entry.get("prompt"),
        }
        cleaned_data.append(cleaned_entry)

    return cleaned_data

loaded_data = load_and_clean_dataset()

In [3]:
# Load OpenFlamingo model
def load_flamingo():
    model, image_processor, tokenizer = create_model_and_transforms(
        clip_vision_encoder_path="ViT-L-14-quickgelu",
        clip_vision_encoder_pretrained="openai",
        lang_encoder_path="facebook/opt-2.7b",
        tokenizer_path="facebook/opt-2.7b"
    )
    return model, image_processor, tokenizer

# Step 2: Extract UI elements using OpenFlamingo
def extract_ui_elements(image_file_path, prompt):
    """
    Uses OpenFlamingo to identify UI elements in an image.
    """
    model, image_processor, tokenizer = load_flamingo()
    print(f"extracting {image_file_path}")
    # Load and process image
    #image = Image.open(image_file_path).convert("RGB")
    image_tensor = image_processor(image).unsqueeze(0)  # Add batch dimension

    #reshape vision_x
    vision_x = image_tensor.unsqueeze(1).unsqueeze(2)

    # Tokenize prompt
    input_text = f"Identify UI elements in this image. {prompt}"
    tokenizer.padding_side = "left"
    lang_x = tokenizer(
        [f"<image>{input_text}<|endofchunk|>"],  
        return_tensors="pt",
    )
    
    print(f"tokenized prompt")

    # Generate response
    with torch.no_grad():
        output = model.generate(
            vision_x=vision_x,  # Now properly shaped
            lang_x=lang_x["input_ids"],
            attention_mask=lang_x["attention_mask"],
            max_length=50
        )
    print(f"extracting elements")
    extracted_elements = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"done extracting")
    return extracted_elements

# Example usage:
image = loaded_data[0]["image"]
prompt = loaded_data[0]["prompt"]
#print(open_clip.list_pretrained())

ui_elements = extract_ui_elements(image, prompt)
print(ui_elements)

/opt/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Flamingo model initialized with 1987192384 trainable parameters
extracting <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1080x1920 at 0x3052C3310>
tokenized prompt


KeyboardInterrupt: 